In [1]:
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import DecimalType
import os
from dotenv import load_dotenv

In [2]:
#Load Environment Variables
load_dotenv()

True

In [3]:
#Setting up credentials

account = os.getenv('account')
user = os.getenv('user')
password = os.getenv('password')
role = os.getenv('role')
warehouse = os.getenv('warehouse')
database = os.getenv('database')
schema = os.getenv('schema')
stage = os.getenv('stage')

In [4]:
#Connecting to snowflake
connection_parameters = {
"account": account,
"user": user,
"password": password,
"role": role,
"warehouse": warehouse,
"database": database,
"schema": schema
 }

#Creating a snowflake session
session = Session.builder.configs(connection_parameters).create()

In [5]:
def minus_one(x: float) -> float:
  return x-1

In [6]:
session.udf.register(
    func = minus_one
  , return_type = DecimalType(20,2)
  , input_types = [DecimalType(20,2)]
  , is_permanent = True
  , name = 'minus_one'
  , replace = True
  , stage_location = stage
 )

In [7]:
#Using SQL with snowpark session
session.sql("""SELECT I_CURRENT_PRICE, MINUS_ONE(I_CURRENT_PRICE) 
                FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.ITEM""").show()

----------------------------------------------------
|"I_CURRENT_PRICE"  |"MINUS_ONE(I_CURRENT_PRICE)"  |
----------------------------------------------------
|4.44               |3.44                          |
|2.10               |1.10                          |
|3.49               |2.49                          |
|3.28               |2.28                          |
|3.19               |2.19                          |
|1.16               |0.16                          |
|9.81               |8.81                          |
|4.81               |3.81                          |
|2.18               |1.18                          |
|74.39              |73.39                         |
----------------------------------------------------

